In [ ]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plot

conn = sqlite3.connect('data/github.db')
data = pd.read_sql_query("""
SELECT strftime('%Y-%m-01', date) AS date, organization, repository,
MAX(stargazers) AS stargazers, MAX(forks) AS forks
FROM github_stats
GROUP BY organization, repository, strftime('%Y-%m', date)
""", conn)

def subset(org_and_repo):
    org, repo = org_and_repo.split('/')
    return data.loc[(data.organization == org) & (data.repository == repo)]

def merge_repos(repos, metric='stargazers'):
    data = pd.DataFrame(columns=['Date'])
    for repo in repos:
        repo_data = subset(repo)
        repo_data = repo_data[['date', metric]].rename(columns={metric: repo, 'date': 'Date'})
        data = data.merge(repo_data, on='Date', how='outer')
    return data.sort_values(by='Date')

def plot_repos(title, repos, metric='stargazers'):
    plot_data = merge_repos(repos, metric)
    plot_data.plot(x='Date', y=repos, title=title, figsize=(15,10))

def plot_all_repos(title, org, metric='stargazers'):
    repo_data = data.loc[data.organization == org]
    repos = [org + '/' + repo for repo in repo_data.repository.unique()]
    plot_repos(title, repos, metric)

In [ ]:
repos = [
    'operator-framework/operator-sdk',
    'kubernetes-sigs/kubebuilder',
    'kubernetes-sigs/controller-runtime',
    'kudobuilder/kudo',
    'bluek8s/kubedirector',
    'kubedb/operator',
    'rook/operator-kit',
    'GoogleCloudPlatform/metacontroller',
    'zalando-incubator/kopf'
]

plot_repos('Operator Tools', repos)

In [ ]:
import requests
import re

r = requests.get('https://twitter.com/kudobuilder')
m = re.search('title="([0-9]+) Followers"', r.text)
followers = int(m.group(1))

cursor = conn.cursor()
cursor.execute('''INSERT INTO twitter_stats (date, followers) VALUES (date('now'), ?)''', (followers,))
conn.commit()

print("@kudobuilder has {} Twitter followers".format(followers))

In [ ]:
plot_all_repos('Operator Framework', 'operator-framework')

In [ ]:
awesome_repo_names = [l.rstrip('\n') for l in open('awesome_operators.txt').readlines()]

def name_and_stars(name):
    awesome_data = subset(name)
    max_stars = awesome_data['stargazers'].max()
    return (name, max_stars)

awesome_data = pd.DataFrame(map(name_and_stars, awesome_repo_names), columns=['repository', 'stargazers']).dropna()

n = 50
top_n_operators = awesome_data.sort_values(by='stargazers', ascending=False).head(n)
top_n_operators

In [ ]:
top_n_operators.plot(x='repository', y='stargazers', kind='bar', figsize=(20,10), title='Github stars for operators listed in awesome-operators')

In [ ]:
repos = [
    'operator-framework/operator-sdk',
    'kubernetes-sigs/kubebuilder',
    'kudobuilder/kudo',
    'bluek8s/kubedirector',
    'kubedb/operator',
    'rook/operator-kit',
    'GoogleCloudPlatform/metacontroller',
    'zalando-incubator/kopf'
]

plot_data = merge_repos(repos, 'forks')
plot_data